# 容器环境 Spark 开发演示

这个 notebook 演示如何在容器化环境中使用 PySpark 进行开发。

## 环境信息
- 运行环境: Docker 容器
- Spark 集群: spark://spark-master:7077
- Python 包: 使用国内镜像源加速安装

In [ ]:
# 导入必要的库
import sys
import os
from datetime import datetime

print(f"Python 版本: {sys.version}")
print(f"当前时间: {datetime.now()}")
print(f"工作目录: {os.getcwd()}")

In [ ]:
# 使用预配置的 PySpark 初始化函数
exec(open('/opt/bitnami/spark/notebooks/pyspark_init.py').read())

# 创建 Spark 会话
spark = create_spark_session("ContainerSparkDemo")

In [ ]:
# 创建示例数据
data = [
    (1, "张三", 25, "北京"),
    (2, "李四", 30, "上海"),
    (3, "王五", 35, "广州"),
    (4, "赵六", 28, "深圳"),
    (5, "钱七", 32, "杭州")
]

columns = ["id", "name", "age", "city"]
df = spark.createDataFrame(data, columns)

print("📊 创建的 DataFrame:")
df.show()

In [ ]:
# DataFrame 基本操作
print("📈 DataFrame 基本信息:")
print(f"行数: {df.count()}")
print(f"列数: {len(df.columns)}")
print(f"列名: {df.columns}")

print("\n📋 Schema 信息:")
df.printSchema()

In [ ]:
# 数据过滤和转换
from pyspark.sql.functions import col, avg, max, min

# 过滤年龄大于 30 的记录
older_people = df.filter(col("age") > 30)
print("🔍 年龄大于 30 的人员:")
older_people.show()

# 计算统计信息
stats = df.agg(
    avg("age").alias("平均年龄"),
    max("age").alias("最大年龄"),
    min("age").alias("最小年龄")
)

print("📊 年龄统计:")
stats.show()

In [ ]:
# 使用 Spark SQL
df.createOrReplaceTempView("people")

# SQL 查询
sql_result = spark.sql("""
    SELECT city, COUNT(*) as count, AVG(age) as avg_age
    FROM people 
    GROUP BY city
    ORDER BY avg_age DESC
""")

print("🏙️ 按城市统计:")
sql_result.show()

In [ ]:
# 数据可视化 (使用 matplotlib)
import matplotlib.pyplot as plt
import pandas as pd

# 转换为 Pandas DataFrame 进行可视化
pandas_df = df.toPandas()

# 创建年龄分布图
plt.figure(figsize=(10, 6))

plt.subplot(1, 2, 1)
plt.bar(pandas_df['name'], pandas_df['age'])
plt.title('年龄分布')
plt.xlabel('姓名')
plt.ylabel('年龄')
plt.xticks(rotation=45)

plt.subplot(1, 2, 2)
city_counts = pandas_df['city'].value_counts()
plt.pie(city_counts.values, labels=city_counts.index, autopct='%1.1f%%')
plt.title('城市分布')

plt.tight_layout()
plt.show()

print("📈 数据可视化完成")

In [ ]:
# 读取外部数据文件 (如果存在)
data_path = "/opt/bitnami/spark/data"
print(f"📁 数据目录内容: {os.listdir(data_path)}")

# 尝试读取 CSV 文件 (如果存在)
try:
    # 这里可以读取你放在 data/ 目录下的文件
    # csv_df = spark.read.csv(f"{data_path}/your_file.csv", header=True, inferSchema=True)
    # csv_df.show(5)
    print("💡 提示: 将你的数据文件放在 data/ 目录下，然后在这里读取")
except Exception as e:
    print(f"📝 注意: {str(e)}")

In [ ]:
# 集群信息和性能监控
sc = spark.sparkContext

print("🖥️  Spark 集群信息:")
print(f"  应用名称: {sc.appName}")
print(f"  应用 ID: {sc.applicationId}")
print(f"  Master URL: {sc.master}")
print(f"  Spark 版本: {sc.version}")
print(f"  默认并行度: {sc.defaultParallelism}")

# 获取 Executor 信息
executors = sc.statusTracker().getExecutorInfos()
print(f"  Executor 数量: {len(executors)}")

for executor in executors:
    print(f"    Executor {executor.executorId}: {executor.host}")

In [ ]:
# 清理资源
spark.stop()
print("✅ Spark 会话已关闭")
print("\n🎉 容器环境 Spark 开发演示完成！")
print("\n💡 下一步:")
print("  1. 探索 notebooks/ 目录下的其他教程")
print("  2. 将你的数据文件放在 data/ 目录下")
print("  3. 开始你的 Spark 学习项目")